In [ ]:
pip install pennylane

In [2]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

symbols = ["Be", "H", "H"]
coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, -1.32, 0.0, 0.0, 1.32])
basis_set = "sto-3g"

H, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    basis=basis_set,
)

In [3]:
electrons = 2
hf = qchem.hf_state(electrons, qubits)

In [4]:
singles, doubles = qchem.excitations(electrons=electrons, orbitals=qubits)
num_theta = len(singles) + len(doubles)

def circuit_VQE(theta, wires):
    qml.AllSinglesDoubles(
        weights = theta,
        wires = wires,
        hf_state = hf,
        singles = singles,
        doubles = doubles)

In [5]:
dev = qml.device('default.qubit', wires=qubits)
@qml.qnode(dev)
def cost_fn(theta):
    circuit_VQE(theta,range(qubits))
    return qml.expval(H)

In [6]:
stepsize = 0.4
max_iterations = 30
opt = qml.GradientDescentOptimizer(stepsize=stepsize)
theta = np.random.uniform(0,1,num_theta, requires_grad=True)

In [ ]:
for n in range(max_iterations):

    theta, prev_energy = opt.step_and_cost(cost_fn, theta)
    samples = cost_fn(theta)

    print('VQE energy: %.4f' %(prev_energy))
    print('Optimal parameters:', theta)

energy_VQE = cost_fn(theta)
theta_opt = theta

print('VQE energy: %.4f' %(energy_VQE))
print('Optimal parameters:', theta_opt)